<a href="https://colab.research.google.com/github/brianisinga/SpamGuard/blob/main/spamguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#

In [5]:
data = pd.read_csv('/content/spam.csv', encoding='latin-1')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## **text cleaning**

In [20]:
import nltk
nltk.download('stopwords')
!pip install emoji
!pip install pyspellchecker

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.5 MB/s eta 0:00:00


In [24]:
import pandas as pd
import re
import string
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import spellchecker

# Load the data
data = pd.read_csv('/content/spam.csv', encoding='latin-1')

# Function to clean the text
def clean_text(text):
    """
    Cleans the input text by removing special characters, converting to lowercase,
    and handling potential NaN values.
    """
    if pd.isna(text):  # Handle NaN values, replace with empty string
        return ''
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ''.join([i for i in text if not i.isdigit()])
    #spell = spellchecker()
    #tokens = [spell.correction(token) for token in tokens]
    text = emoji.demojize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply the cleaning function to all text columns
for column in data.columns:
    if data[column].dtype == 'object':  # Check if column is of text type
        data[column] = data[column].apply(clean_text)

# Display the first few rows of the cleaned data
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,go jurong point crazy available bugis n great ...,,,
1,ham,ok lar joking wif u oni,,,
2,spam,free entry wkly comp win fa cup final tkts st ...,,,
3,ham,u dun say early hor u c already say,,,
4,ham,nah dont think goes usf lives around though,,,


In [27]:
for i in range(0x1F600, 0x1F64F):
    text = emoji.demojize(chr(i))
    print(text, end=" ")
    print(chr(i), end=" ")

:grinning_face: 😀 :beaming_face_with_smiling_eyes: 😁 :face_with_tears_of_joy: 😂 :grinning_face_with_big_eyes: 😃 :grinning_face_with_smiling_eyes: 😄 :grinning_face_with_sweat: 😅 :grinning_squinting_face: 😆 :smiling_face_with_halo: 😇 :smiling_face_with_horns: 😈 :winking_face: 😉 :smiling_face_with_smiling_eyes: 😊 :face_savoring_food: 😋 :relieved_face: 😌 :smiling_face_with_heart-eyes: 😍 :smiling_face_with_sunglasses: 😎 :smirking_face: 😏 :neutral_face: 😐 :expressionless_face: 😑 :unamused_face: 😒 :downcast_face_with_sweat: 😓 :pensive_face: 😔 :confused_face: 😕 :confounded_face: 😖 :kissing_face: 😗 :face_blowing_a_kiss: 😘 :kissing_face_with_smiling_eyes: 😙 :kissing_face_with_closed_eyes: 😚 :face_with_tongue: 😛 :winking_face_with_tongue: 😜 :squinting_face_with_tongue: 😝 :disappointed_face: 😞 :worried_face: 😟 :angry_face: 😠 :enraged_face: 😡 :crying_face: 😢 :persevering_face: 😣 :face_with_steam_from_nose: 😤 :sad_but_relieved_face: 😥 :frowning_face_with_open_mouth: 😦 :anguished_face: 😧 :fearful_fac

😀 😁 😂 😃 😄 😅 😆 😇 😈 😉 😊 😋 😌 😍 😎 😏 😐 😑 😒 😓 😔 😕 😖 😗 😘 😙 😚 😛 😜 😝 😞 😟 😠 😡 😢 😣 😤 😥 😦 😧 😨 😩 😪 😫 😬 😭 😮 😯 😰 😱 😲 😳 😴 😵 😶 😷 😸 😹 😺 😻 😼 😽 😾 😿 🙀 🙁 🙂 🙃 🙄 🙅 🙆 🙇 🙈 🙉 🙊 🙋 🙌 🙍 🙎 